___
# mu7RON Demo 4
---
## Pulses

mu7RON contains some useful tools for adding pulses at specific divisions within a midi sequence.

This notebook gives a short demonstration of how one might go about that.

I wanted to experiment with varying input length of my machine learning models by 'beat' or bar and also wanted to see what would happen if I imbued the input with pulses on certain note divisions etc... So I wrote some functions to enable this:

#### Preparation

In [1]:
import copy

import midi

from MuGen import utils, edit, temporal

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


Below we will load up a midi file, strip it of midi.events objects that are not required and then flatten/consolidate the midi.Tracks into a single midi.Track object:

In [2]:
m = midi.read_midifile(r"mu7ron/data/midi/test/test0.mid")

In [3]:
typs_2_keep = (midi.NoteOnEvent, midi.NoteOffEvent, midi.TimeSignatureEvent, midi.SetTempoEvent)

m = edit.filter_ptrn_of_evnt_typs(m, typs_2_keep)
m = edit.consolidate_trcks(m)

We can see below that the new midi.Pattern contains a single midi.Track with just the midi.events that we specified above with 'types_to_keep'.

In [4]:
print('Type:', type(m))
print('Type:', type(m[0]))
print('No. of tracks:', len(m))
m[0][:10]

Type: <class 'midi.containers.Pattern'>
Type: <class 'midi.containers.Track'>
No. of tracks: 1


midi.Track(\
  [midi.SetTempoEvent(tick=0, data=[6, 102, 252]),
   midi.TimeSignatureEvent(tick=0, data=[4, 2, 6, 102]),
   midi.NoteOnEvent(tick=0, channel=0, data=[84, 79]),
   midi.NoteOnEvent(tick=0, channel=1, data=[81, 79]),
   midi.NoteOnEvent(tick=0, channel=2, data=[77, 79]),
   midi.NoteOnEvent(tick=0, channel=3, data=[69, 79]),
   midi.NoteOnEvent(tick=0, channel=4, data=[60, 79]),
   midi.NoteOnEvent(tick=0, channel=5, data=[53, 79]),
   midi.NoteOffEvent(tick=390, channel=1, data=[81, 80]),
   midi.NoteOnEvent(tick=90, channel=1, data=[81, 79])])

#### Add a bass drum

We can add a pulse to the midi sequence with the `edit.add_pulse` function.

This has as the signature: `edit.add_pulse(midi.Pattern, division, add_pulse_function, division_function)`

Where:

`add_pulse_function` is a user defined function that has the signature:  
`add_pulse_function(alist, tick, on)` where `alist` is the midi sequence currently being edited, `tick` is the tick where the pulse will occur in the midi sequence, `on` is a bool that allows for a pulse/beat to be subdivided by 2 and indicates whether the pulse is 'on' or 'off' beat.

`division_function` is a function that calculates tpp (ticks per pulse). There are two options:
`time.ticks_per_bar_division` or `time.ticks_per_note_division` depending if you want your pulses to be on bar divisions or note divisions respectively.


`division` is a number representing the division e.g. 4 for quarter notes when `division_function` is `time.ticks_per_note_division`.

For example:

In [5]:
c = copy.deepcopy(m) # so we ca re-run cell with a fresh version of m each time

In [6]:
utils.play(c)

Below is our `add_pulse_function`   
It basically just appends a midi.events object to 'alist'. Notice that this is a NoteOnEvent on channel 9, which means it is percussion. And because .data[0] = 35 it is a bass drum!

In [7]:
def add_bass_drum(alist, tick, on, **kwargs):
    alist.append(midi.NoteOnEvent(tick=tick, channel=9, data=[35, 100]))
    return alist

We then apply the function on a pulse with `edit.add_pulse` and play:

In [8]:
c = edit.add_pulse(c, 4., add_bass_drum, temporal.ticks_per_note_division)
utils.play(c)

You should hear a bass drum on each quarter note!

#### Add a drum beat

Below we will add a more complicated drum pattern:

In [9]:
def add_bass_drum(alist, tick, on, **kwargs):
    if on:
        alist.append(midi.NoteOnEvent(tick=tick, channel=9, data=[35, 100]))
    return alist

def add_snare_drum(alist, tick, on, **kwargs):
    if not on:
        alist.append(midi.NoteOnEvent(tick=tick, channel=9, data=[40, 75]))
    return alist

def add_open_hihat_drum(alist, tick, on, **kwargs):
    if not on:
        alist.append(midi.NoteOnEvent(tick=tick, channel=9, data=[53, 75]))
    return alist

def add_closed_hihat_drum(alist, tick, on, **kwargs):
    #if not on:
    alist.append(midi.NoteOnEvent(tick=tick, channel=9, data=[59, 75]))
    return alist

c = copy.copy(m) # so we ca re-run cell with a fresh version of m each time
c = edit.add_pulse(c, 4., add_bass_drum, temporal.ticks_per_note_division)
c = edit.add_pulse(c, 8., add_open_hihat_drum, temporal.ticks_per_note_division)
c = edit.add_pulse(c, 8., add_closed_hihat_drum, temporal.ticks_per_note_division)
c = edit.add_pulse(c, 4, add_snare_drum, temporal.ticks_per_note_division)

utils.play(c)

#### Add a marker


In the `notes` module, there are `Note` classes which can be used as simple place holders for a note division:

In [10]:
from MuGen import notes

In [11]:
c = copy.copy(m) # so we ca re-run cell with a fresh version of m each time

def add_qnote(alist, tick, res, tsig, **kwargs):
    alist.append(notes.QuarterNote(tick=tick, res=res, tsig=tsig))
    return alist

c = edit.add_pulse(c, 4., add_qnote, temporal.ticks_per_note_division)

c[0][:40]

midi.Track(\
  [QuarterNote(tick=0, res=480, tsig=[4, 2]),
   midi.SetTempoEvent(tick=0, data=[6, 102, 252]),
   midi.TimeSignatureEvent(tick=0, data=[4, 2, 6, 102]),
   midi.NoteOnEvent(tick=0, channel=0, data=[84, 79]),
   midi.NoteOnEvent(tick=0, channel=1, data=[81, 79]),
   midi.NoteOnEvent(tick=0, channel=2, data=[77, 79]),
   midi.NoteOnEvent(tick=0, channel=3, data=[69, 79]),
   midi.NoteOnEvent(tick=0, channel=4, data=[60, 79]),
   midi.NoteOnEvent(tick=0, channel=5, data=[53, 79]),
   midi.NoteOffEvent(tick=390, channel=1, data=[81, 80]),
   QuarterNote(tick=90, res=480, tsig=[4, 2]),
   midi.NoteOnEvent(tick=0, channel=1, data=[81, 79]),
   midi.NoteOffEvent(tick=90, channel=0, data=[84, 80]),
   midi.NoteOffEvent(tick=120, channel=1, data=[81, 80]),
   midi.NoteOnEvent(tick=30, channel=0, data=[84, 79]),
   midi.NoteOnEvent(tick=0, channel=1, data=[79, 79]),
   midi.NoteOffEvent(tick=60, channel=2, data=[77, 80]),
   midi.NoteOffEvent(tick=0, channel=3, data=[69, 80]),
   m

These can be used for marking beats for building an AI model